In [1]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf

In [2]:
@load "checkered_all.jl.split02merge02.jld2"
dic1 = copy(dic)

Dict{Any, Any} with 84 entries:
  "lamg_build"        => [328.617, 328.577, 326.042, 323.375, 328.147, 341.11, …
  "acM2C2_err"        => [8.93862e-9, 9.79105e-9, 6.93527e-9, 9.92804e-9, 8.532…
  "hypre_solve"       => [53.05, 52.85, 59.01, 57.76, 51.4, 51.69, 48.71]
  "acM2C2_solve"      => [413.64, 416.629, 501.38, 529.62, 524.068, 541.924, 33…
  "icc2_tot"          => [564.203, 1022.22, 932.083, 1133.57, 1089.89, 1202.5, …
  "acM2C2_tot"        => [700.323, 712.95, 796.936, 819.754, 804.202, 800.276, …
  "ac_lst_build"      => [194.216, 217.025, 215.782, 261.644, 208.624, 201.018,…
  "petsc_hypre_build" => [354.743, 427.164, 417.284, 378.951, 320.113, 183.512,…
  "lamg_its"          => [8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0]
  "lamg_solve"        => [360.299, 356.768, 355.728, 350.849, 352.721, 375.776,…
  "hypre_its"         => [8.0, 8.0, 9.0, 9.0, 8.0, 8.0, 7.0]
  "cmg_solve"         => [84.34, 87.1662, 85.0853, 89.2418, 79.7099, 76.6486, 8…
  "ac_tot"            => [376.584, 419.09, 

In [3]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        

#constant d = 3
ff(s) = (@sprintf "%.3g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))

ffK(s) = (@sprintf "%.3g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)

ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)

ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)

ffe6 (generic function with 1 method)

In [4]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printSolverCell(d, solver, i)
    if haskey(d, string(solver, "_tot"))
        itr = d[string(solver, "_its")][i]
        ne = d["ne"][i]
        solveVn = d[string(solver, "_solve")][i] / ne
        totalVn = d[string(solver, "_tot")][i] / ne
        err = d[string(solver, "_err")][i]
        return string("&", nf(itr), "&", labelTvn(solveVn, err), "&", labelTvn(totalVn, err), "&",ffe8(err))
    else
        return "&N/A&N/A&N/A&N/A"
    end
end

function printOneLine(d, solvers, i)
    allCells = [printSolverCell(d, solver, i) for solver in solvers]
    return string(2^i, join(allCells), "\\\\")
end

function printAllLines(d, solvers)
    for i in 1:length(d["testName"])
        println(printOneLine(d, solvers, i))
    end
end

printAllLines (generic function with 1 method)

In [5]:
solvers1 = ["ac", "ac-s2m2", "cmg2"]
printAllLines(dic1, solvers1)

2&41&1.28&1.88&0.742&30&1.48&3.01&0.987&30&0.668&1.07&0.643\\
4&47&1.47&2.1&0.733&35&1.74&3.14&0.786&28&0.569&0.981&0.796\\
8&52&1.61&2.24&0.836&38&1.87&3.26&0.985&26&0.531&0.937&0.912\\
16&52&1.57&2.19&0.915&46&2.2&3.57&0.707&28&0.571&1&0.969\\
32&59&1.73&2.33&0.802&45&2.06&3.38&0.967&28&0.552&0.971&0.718\\
64&57&1.59&2.16&0.925&48&2.02&3.23&0.762&25&0.506&0.93&0.671\\
128&53&1.39&1.94&0.98&35&1.31&2.46&0.828&28&0.519&0.956&0.989\\


In [6]:
solvers2 = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dic1, solvers2)

2&8&0.265&0.616&0.308&7&1.35&3.12&0.000145&172&2.67&2.82&0.98\\
4&8&0.264&0.656&0.738&7&1.65&3.79&0.000555&321&4.96&5.11&0.969\\
8&9&0.295&0.734&0.351&7&1.77&3.85&0.00142&292&4.51&4.66&0.922\\
16&9&0.289&0.752&0.184&8&1.95&3.84&0.000297&358&5.51&5.67&0.845\\
32&8&0.257&0.732&0.744&8&1.83&3.43&0.00117&343&5.3&5.45&0.916\\
64&8&0.258&0.64&0.513&9&1.43&2.35&0.000498&380&5.86&6.01&0.947\\
128&7&0.244&0.55&0.98&7&0.664&1.1&0.00106&413&6.37&6.52&0.965\\
